<a href="https://colab.research.google.com/github/DanialQureshi26/ML-Projects/blob/main/Skin_Disease_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

# Define the directory containing your dataset
dataset_dir = '/content/drive/My Drive/skin disease'

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create an ImageDataGenerator instance for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
)

# Get the list of all image file paths and their corresponding labels
data = []
labels = []
for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    for img_file in os.listdir(category_path):
        img_path = os.path.join(category_path, img_file)
        data.append(img_path)
        labels.append(category)

# Convert to DataFrame
df = pd.DataFrame({
    'filename': data,
    'class': labels
})

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])

# Create data generators
train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-4:]:  # Unfreeze the last 4 layers
    layer.trainable = True

# Create a new model on top of the pre-trained base model
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(os.listdir(dataset_dir)), activation='softmax')
])

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for learning rate adjustment and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Mounted at /content/drive
Found 739 validated image filenames belonging to 8 classes.
Found 185 validated image filenames belonging to 8 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


58889256/58889256 [==============================] - 1s 0us/step
Epoch 1/20
23/23 [==============================] - 286s 11s/step - loss: 2.1506 - accuracy: 0.1386 - val_loss: 2.0866 - val_accuracy: 0.1437 - lr: 1.0000e-04
Epoch 2/20
23/23 [==============================] - 16s 678ms/step - loss: 2.0739 - accuracy: 0.1641 - val_loss: 2.0217 - val_accuracy: 0.1500 - lr: 1.0000e-04
Epoch 3/20
23/23 [==============================] - 16s 683ms/step - loss: 2.0416 - accuracy: 0.1655 - val_loss: 1.8215 - val_accuracy: 0.3125 - lr: 1.0000e-04
Epoch 4/20
23/23 [==============================] - 16s 669ms/step - loss: 2.0090 - accuracy: 0.1909 - val_loss: 1.8707 - val_accuracy: 0.3125 - lr: 1.0000e-04
Epoch 5/20
23/23 [==============================] - 16s 676ms/step - loss: 1.8618 - accuracy: 0.2843 - val_loss: 1.7094 - val_accuracy: 0.3438 - lr: 1.0000e-04
Epoch 6/20
23/23 [==============================] - 16s 682ms/step - loss: 1.8416 - accuracy: 0.2871 - val_loss: 1.5026 - val_accuracy: 

In [4]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load and preprocess the image
def preprocess_image(image_path, img_width, img_height):
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Scale pixel values to [0, 1]
    return img_array

# Function to predict the disease from an image
def predict_disease(image_path, model, class_indices):
    # Preprocess the image
    img_array = preprocess_image(image_path, img_width, img_height)

    # Make a prediction
    predictions = model.predict(img_array)

    # Get the predicted class index
    predicted_class_idx = np.argmax(predictions, axis=1)[0]

    # Map the predicted class index to the disease name
    class_labels = {v: k for k, v in class_indices.items()}
    predicted_class = class_labels[predicted_class_idx]

    return predicted_class

# Example usage
image_path = '/content/drive/MyDrive/test image 3.jpg'  # Replace with the path to your image
predicted_disease = predict_disease(image_path, model, train_generator.class_indices)
print("Predicted disease:", predicted_disease)


1/1 [==============================] - 0s 18ms/step
Predicted disease: VI-shingles


In [5]:
# prompt: save the above mode

model.save('/content/drive/MyDrive/my_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
